In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import *
import nltk
import json
import pandas as pd
import numpy as np
import torch

SEED = 42
nltk.download('punkt')

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
model_path = "../../models"

In [3]:
# model to fine tune
model_checkpoint = "cjvt/t5-sl-small"
# model_checkpoint = "cjvt/t5-sl-large"
# model_checkpoint = "google/mt5-small"
# model_checkpoint = "google/mt5-base"
# model_checkpoint = "google/mt5-large"
name = model_checkpoint.split("/")[-1]

epochs = 10

# this batch size fits on a GPU with 12GB of RAM with the model_checkpoint = "cjvt/t5-sl-small"
batch_size = 16
# this batch size fits on a GPU with 12GB of RAM with the model_checkpoint = "google/mt5-small"
# batch_size = 8

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

### Load the data

In [4]:
data = pd.read_csv("../../data/processed/prompt_reply_pairs.csv", sep=";")
data

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
47509,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
47510,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
47511,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
47512,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test


In [5]:
train_data = data[data["split"] == "train"]
val_data = data[data["split"] == "val"]
test_data = data[data["split"] == "test"]
len(train_data), len(val_data), len(test_data)

(37902, 4737, 4875)

In [6]:
train_data = Dataset.from_pandas(train_data[['prompt', 'reply']])
val_data = Dataset.from_pandas(val_data[['prompt', 'reply']])
test_data = Dataset.from_pandas(test_data[['prompt', 'reply']])
train_data

Dataset({
    features: ['prompt', 'reply', '__index_level_0__'],
    num_rows: 37902
})

In [7]:
def convert_to_features(examples):
    prefix_in = "Uporabnik: "
    # prefix_in = ""
    examples["prompt"] = [prefix_in + prompt for prompt in examples["prompt"]]
    # prefix_out = "Asistent: "
    prefix_out = ""
    examples["reply"] = [prefix_out + reply for reply in examples["reply"]]
    
    model_inputs = tokenizer(examples['prompt'], pad_to_max_length=True, max_length=512, truncation=True, return_tensors='pt')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reply'], pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
train_data = train_data.map(convert_to_features, batched=True, load_from_cache_file=False)
val_data = val_data.map(convert_to_features, batched=True, load_from_cache_file=False)
test_data = test_data.map(convert_to_features, batched=True, load_from_cache_file=False)
train_data

Map:   0%|          | 0/37902 [00:00<?, ? examples/s]c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.w

Dataset({
    features: ['prompt', 'reply', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 37902
})

### Fine Tune

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_path}/{name}-finetuned-assistant",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=False, # setting this to true gives loss 0.0 at every step for some reason
    push_to_hub=False, 
    load_best_model_at_end=True # load best model at end so we save the best model instead of the last model
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()
trainer.save_model(f"{model_path}/{name}-finetuned-assistant")

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 21%|██        | 500/2369 [01:55<07:12,  4.32it/s]

{'loss': 9.6262, 'learning_rate': 1.5778809624314057e-05, 'epoch': 0.21}


 33%|███▎      | 772/2369 [03:00<06:06,  4.35it/s]

KeyboardInterrupt: 

### Evaluate

Simple example

In [ ]:
input = "Uporabnik: Kdo je France Prešeren?"
input = tokenizer(input, return_tensors="pt").to("cuda")
outputs = trainer.model.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Validation set

In [ ]:
val_results = trainer.evaluate()
print('Val results: ', val_results)

Test set

In [ ]:
test_results = trainer.predict(test_dataset=test_data)
print('Test results:', test_results.metrics)